In [121]:
# from google.colab import drive
# drive.mount('/content/drive')

In [122]:
# !unzip "/content/drive/MyDrive/archive(1).zip" -d "/content/"

In [123]:
!pip install nibabel
!pip install nilearn

import sys
import os

import numpy as np
import tensorflow as tf
import pandas as pd
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from skimage.util import montage
from skimage.transform import rotate,resize
from sklearn.model_selection import train_test_split
from keras.src import Input
from keras.src.models import Model
from keras.src.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.src.metrics import Precision,Recall
import cv2
import random
import keras

In [1]:
def is_empty(image):
    return np.all(image == 0)

def load_nii_file(filepath,mx = 0):
    nii_file = nib.load(filepath)
    data = np.array(nii_file.get_fdata(), dtype=np.float32)

    if is_empty(data):
        return data

    if mx <= 0:
        data = (data-np.min(data)) / (np.max(data)-np.min(data))
        brain = data > 0
        mean = data[brain].mean()
        std = data[brain].std()
        data = (data - mean) / std
    else:
        data = data/mx

    data = data[:, :, 14:-13]

    return data


def load_nii_dataset(input_filepaths, output_filepaths):
    def generator():
        for input_file, output_file in zip(input_filepaths, output_filepaths):

            flair_nii = load_nii_file(input_file[0])
            t1ce_nii = load_nii_file(input_file[1])
            t1_nii = load_nii_file(input_file[2])
            t2_nii = load_nii_file(input_file[3])

            output_nii = load_nii_file(output_file,mx=4)
            output_nii = np.floor(output_nii*4)
            output_nii = np.where(output_nii == 4, 3, output_nii)

            flair_nii = tf.convert_to_tensor(flair_nii, dtype=tf.float32)
            t1ce_nii = tf.convert_to_tensor(t1ce_nii, dtype=tf.float32)
            t1_nii = tf.convert_to_tensor(t1_nii, dtype=tf.float32)
            t2_nii  = tf.convert_to_tensor(t2_nii , dtype=tf.float32)

            output_nii = tf.convert_to_tensor(output_nii, dtype=tf.int32)

            flair_nii = tf.expand_dims(flair_nii, axis=-1)
            t1ce_nii = tf.expand_dims(t1ce_nii, axis=-1)
            t1_nii = tf.expand_dims(t1_nii, axis=-1)
            t2_nii = tf.expand_dims(t2_nii, axis=-1)

            input_combined = tf.concat([flair_nii, t1ce_nii, t1_nii, t2_nii], axis=-1)

            output_nii = tf.one_hot(output_nii, depth=4)

            yield input_combined, output_nii

    dataset = tf.data.Dataset.from_generator(
        generator=generator,
        output_signature=(
            tf.TensorSpec(shape=(240,240,128,4), dtype=tf.float32),
            tf.TensorSpec(shape=(240,240,128,4), dtype=tf.float32)
        )
    )

    return dataset


In [125]:
from keras.src.layers import Add,Multiply,ZeroPadding2D,Dropout,ZeroPadding3D, BatchNormalization, LeakyReLU, ReLU, Conv2D, MaxPooling2D, UpSampling2D,  Conv3D, MaxPooling3D, UpSampling3D,concatenate, Input
from keras.src.models import Model

activations = {
    'relu': ReLU,
    'leaky_relu': LeakyReLU,
}


def Reshape3D(original_block,up):
    if up.shape[1] != original_block.shape[1] or up.shape[2] != original_block.shape[2] or up.shape[3] != original_block.shape[3]:
        diff_depth = original_block.shape[1] - up.shape[1]
        diff_height = original_block.shape[2] - up.shape[2]
        diff_width = original_block.shape[3] - up.shape[3]

        up = ZeroPadding3D(((diff_depth // 2, diff_depth - diff_depth // 2),
            (diff_height // 2, diff_height - diff_height // 2),
            (diff_width // 2, diff_width - diff_width // 2))
        )(up)
    return up

def Reshape2D(original_block, up):
    if up.shape[1] != original_block.shape[1] or up.shape[2] != original_block.shape[2]:
        diff_height = original_block.shape[1] - up.shape[1]
        diff_width = original_block.shape[2] - up.shape[2]

        up = ZeroPadding2D(((diff_height // 2, diff_height - diff_height // 2),
                            (diff_width // 2, diff_width - diff_width // 2)))(up)
    return up

def IdentityBlock(x):

    return x

def Normalize(x, normalize):

    return BatchNormalization()(x) if normalize else x

def DropOut(x, drop_out):

    return Dropout(drop_out)(x) if 0.0 < drop_out < 1.0 else x

def EncoderBlock(input_block, filter_size, kernel_size=3, padding="same",
                 activation="relu", slope=0.0, pool_size=(2,2),
                 batch_normalization=False, drop_out=0.0,kernel_regularizer=None):

    conv = Conv2D(filters=filter_size,kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(input_block)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv2D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    pool = MaxPooling2D(pool_size=pool_size)(conv)
    pool = DropOut(pool, drop_out)

    return conv, pool

def BottleneckBlock(input_block, filter_size, kernel_size=3, padding="same",
                    activation="relu", slope=0.0, batch_normalization=False,kernel_regularizer=None):

    conv = Conv2D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(input_block)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv2D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    return conv

def DecoderBlock(input_block, original_block, filter_size, kernel_size=3, padding="same",
                 activation="relu", slope=0.0, batch_normalization=False, drop_out=0.0,kernel_regularizer=None):

    up = UpSampling2D(size=(2,2))(input_block)
    up = Reshape2D(original_block=original_block,up = up)
    merge = concatenate([original_block, up], axis=3)

    conv = Conv2D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(merge)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv2D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    return conv

def OutputBlock(input_block,num_of_classes = 1,kernel_size=1,activation = "sigmoid",padding="same",kernel_regularizer=None):
    return Conv2D(filters=num_of_classes,kernel_size=kernel_size,activation=activation,padding=padding,kernel_regularizer=kernel_regularizer)(input_block)



def EncoderBlock3D(input_block, filter_size, kernel_size=3, padding="same",
                 activation="relu", slope=0.0, pool_size=(2, 2, 2),
                 batch_normalization=False, drop_out=0.0,kernel_regularizer=None):

    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(input_block)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    pool = MaxPooling3D(pool_size=pool_size)(conv)
    pool = DropOut(pool, drop_out)

    return conv, pool

def BottleneckBlock3D(input_block, filter_size, kernel_size=3, padding="same",
                    activation="relu", slope=0.0, batch_normalization=False,kernel_regularizer=None):
    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(input_block)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    return conv

def AttentionDecoder(input_block, original_block, filter_size, kernel_size=3, padding="same",
                    activation="relu", slope=0.0, batch_normalization=False,kernel_regularizer=None):

    up = UpSampling2D(size=(2, 2))(input_block)
    up = Reshape2D(original_block=original_block, up=up)

    theta_x = Conv2D(filter_size, kernel_size=kernel_size, strides=1, padding=padding)(original_block)
    phi_g = Conv2D(filter_size, kernel_size=kernel_size, strides=1, padding=padding)(up)

    attention = Add()([theta_x, phi_g])
    attention = Normalize(attention, batch_normalization)
    attention = activations.get(activation, ReLU)(negative_slope=slope)(attention)
    attention = Conv2D(1, kernel_size=kernel_size, activation='sigmoid', padding=padding,kernel_regularizer=kernel_regularizer)(attention)

    attention = Multiply()([original_block, attention])

    conv = Conv2D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(attention)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv2D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    return conv



def AttentionDecoder3D(input_block, original_block, filter_size, kernel_size=3, padding="same",
                    activation="relu", slope=0.0, batch_normalization=False,kernel_regularizer=None,drop_out=0):

    up = UpSampling3D(size=(2, 2, 2))(input_block)
    up = Reshape3D(original_block=original_block, up=up)

    theta_x = Conv3D(filter_size, kernel_size=kernel_size, strides=1, padding=padding)(original_block)
    phi_g = Conv3D(filter_size, kernel_size=kernel_size, strides=1, padding=padding)(up)

    attention = Add()([theta_x, phi_g])
    attention = Normalize(attention, batch_normalization)
    attention = activations.get(activation, ReLU)(negative_slope=slope)(attention)
    attention = Conv3D(1, kernel_size=kernel_size, activation='sigmoid', padding=padding,kernel_regularizer=kernel_regularizer)(attention)

    attention = Multiply()([original_block, attention])

    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(attention)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding,kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = DropOut(conv, drop_out)

    return conv


def DecoderBlock3D(input_block, original_block, filter_size, kernel_size=3, padding="same",
                 activation="relu", slope=0.0, batch_normalization=False, drop_out=0.0,kernel_regularizer=None):

    up = UpSampling3D(size=(2, 2, 2))(input_block)
    up = Reshape3D(original_block=original_block,up=up)
    merge = concatenate([original_block, up], axis=-1)

    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding, kernel_regularizer=kernel_regularizer)(merge)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    conv = Conv3D(filters=filter_size, kernel_size=kernel_size, padding=padding, kernel_regularizer=kernel_regularizer)(conv)
    conv = Normalize(conv, batch_normalization)
    conv = activations.get(activation, ReLU)(negative_slope=slope)(conv)

    return conv

def OutputBlock3D(input_block, num_of_classes=1, kernel_size=1, activation="sigmoid", padding="same",kernel_regularizer=None):
    return Conv3D(filters=num_of_classes, kernel_size=kernel_size, activation=activation, padding=padding, kernel_regularizer=kernel_regularizer)(input_block)

def U_NET(inputs,outputs):
    return Model(inputs,outputs)

In [126]:
from keras.src.saving import register_keras_serializable

@register_keras_serializable()
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)

    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

@register_keras_serializable()
def dice_coef_background(y_true, y_pred, epsilon=1e-6):
    intersection = tf.reduce_sum(tf.abs(y_true[:,:,:,:,0] * y_pred[:,:,:,:,0]), axis=[1, 2, 3])
    return (2. * intersection) / (tf.reduce_sum(tf.square(y_true[:,:,:,:,0]), axis=[1, 2, 3]) + tf.reduce_sum(tf.square(y_pred[:,:,:,:,0]), axis=[1, 2, 3]) + epsilon)

@register_keras_serializable()
def dice_coef_necrotic(y_true, y_pred, epsilon=1e-6):
    intersection = tf.reduce_sum(tf.abs(y_true[:,:,:,:,1] * y_pred[:,:,:,:,1]), axis=[1, 2, 3])
    return (2. * intersection) / (tf.reduce_sum(tf.square(y_true[:,:,:,:,1]), axis=[1, 2, 3]) + tf.reduce_sum(tf.square(y_pred[:,:,:,:,1]), axis=[1, 2, 3]) + epsilon)

@register_keras_serializable()
def dice_coef_edema(y_true, y_pred, epsilon=1e-6):
    intersection = tf.reduce_sum(tf.abs(y_true[:,:,:,:,2] * y_pred[:,:,:,:,2]), axis=[1, 2, 3])
    return (2. * intersection) / (tf.reduce_sum(tf.square(y_true[:,:,:,:,2]), axis=[1, 2, 3]) + tf.reduce_sum(tf.square(y_pred[:,:,:,:,2]), axis=[1, 2, 3]) + epsilon)

@register_keras_serializable()
def dice_coef_enhancing(y_true, y_pred, epsilon=1e-6):
    intersection = tf.reduce_sum(tf.abs(y_true[:,:,:,:,3] * y_pred[:,:,:,:,3]), axis=[1, 2, 3])  # sum over depth, height, width
    return (2. * intersection) / (tf.reduce_sum(tf.square(y_true[:,:,:,:,3]), axis=[1, 2, 3]) + tf.reduce_sum(tf.square(y_pred[:,:,:,:,3]), axis=[1, 2, 3]) + epsilon)

@register_keras_serializable()
def multi_class_dice_loss(y_true, y_pred, smooth=1e-6):
    dice_loss_total = 0.0
    #weights = [0.00431145,0.46406451,0.15163813,0.37998591]
    weights = [0.1,1.0,0.4,1.0]
    num_classes = y_pred.shape[-1]

    for class_idx in range(num_classes):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]

        dice_coef = dice_coefficient(y_true_class, y_pred_class, smooth)
        dice_loss_total += (1 - dice_coef)*weights[class_idx]

    return dice_loss_total / num_classes

@register_keras_serializable()
def ajusted_multi_dice(y_true, y_pred, smooth=1e-6):
    dice_loss_total = 0.0
    #weights = [0.00431145,0.46406451,0.15163813,0.37998591]
    weights = [0.05,0.42,0.21,0.32]
    # weights = [0.01,0.402,0.288,0.30]
    dice_func = [dice_coef_background,dice_coef_necrotic,dice_coef_edema,dice_coef_enhancing]
    num_classes = y_pred.shape[-1]

    for class_idx in range(num_classes):
        dice_coef = dice_func[class_idx](y_true, y_pred, smooth)
        dice_loss_total += (1 - dice_coef)*weights[class_idx]

    return dice_loss_total / num_classes

@register_keras_serializable()
def containment_penalty(y_pred):
    class_1 = y_pred[..., 1]
    class_2 = y_pred[..., 2]
    class_3 = y_pred[..., 3]

    kernel = tf.ones((3, 3, 3, 1, 1), dtype=tf.float32)
    strides = (1, 1, 1, 1, 1)
    padding = 'SAME'

    c2_exp = tf.expand_dims(class_2, -1)
    c3_exp = tf.expand_dims(class_3, -1)

    class2_shell = tf.nn.max_pool3d(c2_exp, ksize=3, strides=1, padding='SAME')
    class3_shell = tf.nn.max_pool3d(c3_exp, ksize=3, strides=1, padding='SAME')

    class2_shell = tf.squeeze(class2_shell, -1)
    class3_shell = tf.squeeze(class3_shell, -1)

    penalty_1 = tf.nn.relu(class_1 - class2_shell)

    mask_1_exists = tf.cast(tf.reduce_sum(class_1, axis=[1,2,3]) > 0, tf.float32)
    mask_1_exists = tf.reshape(mask_1_exists, [-1,1,1,1])

    target = class_1 * mask_1_exists + class_2 * (1 - mask_1_exists)
    penalty_2 = tf.nn.relu(target - class3_shell)

    penalty = tf.reduce_mean(penalty_1 + penalty_2)

    return penalty

@register_keras_serializable()
def combined_loss(y_true, y_pred):
    dice = ajusted_multi_dice(y_true, y_pred)
    cce = keras.losses.CategoricalCrossentropy()
    penalty = containment_penalty(y_pred)
    return 0.65 * dice + 0.35 * cce(y_true, y_pred) + 0.1 * penalty

@register_keras_serializable()
def penalty_dice_loss(y_true, y_pred):
    dice_loss = 1 - ajusted_multi_dice(y_true, y_pred)
    penalty = containment_penalty(y_pred)
    return 0.8*dice_loss - 0.2*penalty


In [127]:
train_path = '/content/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
input_files = []
output_files = []

def getFile(fileList,fileType):
    for i in fileList:
        if i.find(fileType) != -1:
            return i
    print(fileList)
    raise FileNotFoundError("File not found")


In [128]:
for folder in os.listdir(train_path):
    try:
     folder_path = os.path.join(train_path,folder)
     fileList = os.listdir(folder_path)


     input_file = [
         getFile(fileList,"_flair."),
         getFile(fileList,"_t1ce."),
         getFile(fileList,"_t1."),
         getFile(fileList,"_t2.")
     ]
     output_file = getFile(fileList,"_seg.")

     input_files.append([os.path.join(folder_path, f) for f in input_file])
     output_files.append(os.path.join(folder_path,output_file))
    except:
        print("Bad file format")
        print(fileList)

['BraTS20_Training_355_t1ce.nii', 'BraTS20_Training_355_t1.nii', 'BraTS20_Training_355_t2.nii', 'W39_1998.09.19_Segm.nii', 'BraTS20_Training_355_flair.nii']
Bad file format
['BraTS20_Training_355_t1ce.nii', 'BraTS20_Training_355_t1.nii', 'BraTS20_Training_355_t2.nii', 'W39_1998.09.19_Segm.nii', 'BraTS20_Training_355_flair.nii']
Bad file format
['BraTS20_Training_034_flair.nii', 'BraTS20_Training_034_seg.nii', 'BraTS20_Training_034_t2.nii', 'BraTS20_Training_034_t1ce.nii', 'BraTS20_Training_034_t1.nii']
Bad file format
['BraTS20_Training_350_t1.nii', 'BraTS20_Training_350_t1ce.nii', 'BraTS20_Training_350_seg.nii', 'BraTS20_Training_350_t2.nii', 'BraTS20_Training_350_flair.nii']


In [129]:
X_train,X_validation,y_train,y_validation = train_test_split(input_files,output_files,train_size=0.82,random_state=42)

train_dataset = load_nii_dataset(X_train,y_train)
val_dataset = load_nii_dataset(X_validation,y_validation)

train_dataset = train_dataset.batch(2).prefetch(1)
val_dataset = val_dataset.batch(2).prefetch(1)

In [130]:
from keras.src import Input
from keras.src.models import Model
from keras.src.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.src.regularizers import L2

inputs = Input(shape=(240,240,128,4))

conv1,pool1 = EncoderBlock3D(inputs,filter_size=32,activation="leaky_relu",kernel_regularizer=L2(0.0001))
conv2,pool2 = EncoderBlock3D(pool1,filter_size=64,activation="leaky_relu",kernel_regularizer=L2(0.0001))
conv3,pool3 = EncoderBlock3D(pool2,filter_size=128,activation="leaky_relu",kernel_regularizer=L2(0.0001))

neck = BottleneckBlock3D(pool3,filter_size=256,activation="leaky_relu",kernel_regularizer=L2(0.0001))

up1 =  AttentionDecoder3D(neck,conv3,filter_size=128,activation="leaky_relu",kernel_regularizer=L2(0.0001))
up2 =   AttentionDecoder3D(up1,conv2,filter_size=64,activation="leaky_relu",kernel_regularizer=L2(0.0001))
up3 =   AttentionDecoder3D(up2,conv1,filter_size=32,activation="leaky_relu",kernel_regularizer=L2(0.0001))

outputs = OutputBlock3D(up3,num_of_classes=4,kernel_size=1,activation='softmax',kernel_regularizer=L2(0.0001))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model = U_NET(inputs=inputs,outputs=outputs)
model.compile(
     optimizer='adam',
     loss=combined_loss,
     metrics=[dice_coefficient,dice_coef_background,dice_coef_necrotic,dice_coef_edema,dice_coef_enhancing]
)

model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 128, 4)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_246 (Conv3D) │ (None, 240, 240,  │      3,488 │ input_layer_11[0… │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_176     │ (None, 240, 240,  │          0 │ conv3d_246[0][0]  │
│ (LeakyReLU)         │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_247 (Conv3D) │ (None, 240, 240,  │     27,680 │ leaky_re_lu_176[… │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_177     │ (None, 240, 240,  │          0 │ conv3d_247[0][0]  │
│ (LeakyReLU)         │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_33    │ (None, 120, 120,  │          0 │ leaky_re_lu_177[… │
│ (MaxPooling3D)      │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 120, 120,  │          0 │ max_pooling3d_33… │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_248 (Conv3D) │ (None, 120, 120,  │     55,360 │ dropout_3[0][0]   │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_178     │ (None, 120, 120,  │          0 │ conv3d_248[0][0]  │
│ (LeakyReLU)         │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_249 (Conv3D) │ (None, 120, 120,  │    110,656 │ leaky_re_lu_178[… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_179     │ (None, 120, 120,  │          0 │ conv3d_249[0][0]  │
│ (LeakyReLU)         │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_34    │ (None, 60, 60,    │          0 │ leaky_re_lu_179[… │
│ (MaxPooling3D)      │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 60, 60,    │          0 │ max_pooling3d_34… │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_250 (Conv3D) │ (None, 60, 60,    │    221,312 │ dropout_4[0][0]   │
│                     │ 32, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_180     │ (None, 60, 60,    │          0 │ conv3d_250[0][0]  │
│ (LeakyReLU)         │ 32, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_251 (Conv3D) │ (None, 60, 60,    │    442,496 │ leaky_re_lu_180[… │
│                     │ 32, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_181     │ (None, 60, 60,    │          0 │ conv3d_251[0][0]

 Total params: 6,425,831 (24.51 MB)

 Trainable params: 6,425,831 (24.51 MB)

 Non-trainable params: 0 (0.00 B)

In [131]:
early_stopping = EarlyStopping(monitor="val_loss", patience=25, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Keras/brats_3d.keras', save_best_only=True, monitor="val_loss")
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.35, patience=8, min_lr=1e-12)

history = model.fit(
    train_dataset,
    epochs=120,
    steps_per_epoch = len(X_train)//2,
    validation_data=val_dataset,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

Epoch 1/120
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - dice_coef_background: 0.9723 - dice_coef_edema: 0.4054 - dice_coef_enhancing: 0.3826 - dice_coef_necrotic: 0.2252 - dice_coefficient: 0.9412 - loss: 0.4602

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


150/150 ━━━━━━━━━━━━━━━━━━━━ 457s 3s/step - dice_coef_background: 0.9724 - dice_coef_edema: 0.4060 - dice_coef_enhancing: 0.3834 - dice_coef_necrotic: 0.2256 - dice_coefficient: 0.9414 - loss: 0.4588 - val_dice_coef_background: 0.9972 - val_dice_coef_edema: 0.5750 - val_dice_coef_enhancing: 0.6201 - val_dice_coef_necrotic: 0.3723 - val_dice_coefficient: 0.9893 - val_loss: 0.1313 - learning_rate: 0.0010
Epoch 2/120
150/150 ━━━━━━━━━━━━━━━━━━━━ 85s 493ms/step - dice_coef_background: 0.9980 - dice_coef_edema: 0.2385 - dice_coef_enhancing: 0.8412 - dice_coef_necrotic: 0.3717 - dice_coefficient: 0.9909 - loss: 0.1349 - val_dice_coef_background: 0.9972 - val_dice_coef_edema: 0.5574 - val_dice_coef_enhancing: 0.6225 - val_dice_coef_necrotic: 0.3721 - val_dice_coefficient: 0.9893 - val_loss: 0.1320 - learning_rate: 0.0010
Epoch 3/120
150/150 ━━━━━━━━━━━━━━━━━━━━ 432s 3s/step - dice_coef_background: 0.9967 - dice_coef_edema: 0.5790 - dice_coef_enhancing: 0.6357 - dice_coef_necrotic: 0.3810 - di

KeyboardInterrupt: 

In [ ]:
import os
import time

time.sleep(5)

os._exit(0)
